# Topic Guardrail

![Solution Architecture](./images/w_topical.png)

## Topical Rail - Demo
Developing chatbots that stay on topic can be challenging. The key is to build the bot to be knowledgeable about specific subjects.

In this example, we'll look at some ways to keep a chatbot on topic, including:

- Constructing the chatbot knowledge to be focused on particular topics and conversations 
- Providing a quick overview of how to launch a chatbot with mechanisms to keep it on a single topic
- Demonstrating dialogues that illustrate the chatbot sticking to certain subjects and avoiding others

The goal is to guide the chatbot to stay on target when users ask questions, rather than letting the conversation drift aimlessly. By designing the chatbot well and giving it the right scopes of knowledge, we can create more useful and effective conversational agents.  



### How does NeMo Guardrails works for topical identification (in a nutshell)
Topical rail ensures AI responses stay within the predefined topic, and prevents off-topic conversations that has no business value.
1. As part of Guardrails configuration, you define different conversation flow and how Guardrails treat them.
2. You provide example input texts for each flow.
3. When the user sends text, Guardrails intercepts it, and tries to understand which flow it maps to (Embedding based similarity search vs each flow's examples texts). 
4. With the flow established Guardrails carries out the flow (e.g., let's the LLM respond, or replies back the conversation is off-topic).

In [ ]:
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
import json
import boto3
from utils import bedrock, print_ww

# This helper function encompasses the process of initializing NeMo Guardrails and generating Rails based on a specified configuration.
from nemoguardrails import LLMRails, RailsConfig

# BedrockModels is a "Singleton" class which initializes the necessary models for the notebook.
from models import BedrockModels

## Initalize bedrock client

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True,
)

## Define function to create a LLMRails instance with Claudev2 as LLM and a certain rail configuration

In [ ]:
# This helper function encapsulates the necessary steps to bootstrap
# NeMo Guardrails and returns Rails based on a given configuration.
def bootstrap_bedrock_nemo_guardrails(rail_config_path: str) -> LLMRails:

    # 1. initialize rails config
    config = RailsConfig.from_path(f"NeMo/rails/{rail_config_path}/config")

    # initialize bedrock models
    # you can pass model id as string or use the default model id 'anthropic.claude-v2'
    bedrock_models = BedrockModels
    bedrock_models.init_bedrock_client(boto3_bedrock)
    bedrock_models.init_llm("anthropic.claude-v2")

    # 2. bootstraps NeMo Guardrails with the necessary resources
    app = LLMRails(config=config, llm=bedrock_models.llm, verbose=False)

    return app

## rail configuration

To view the full `on-topic.co` configuration content, run the following cell:

In [ ]:
!cat ./NeMo/rails/topical/config/on-topic.co

## Task 1: Define the off-topic.co script

- try to understand the colang structure of the on-topic.co script
- use the same structure to define a colang script that prevents the model to speak about politics (off-topic.co)
- Use the 3 primary building elements "define user", "define bot", "define flow" and then add the utterances as needed

To view the full `off-topic.co` configuration content, run the following cell:

In [ ]:
!cat ./NeMo/rails/topical/config/off-topic.co

## Test the rail

In [ ]:
# Bootstrap Guardrails with topical configuration
rails = bootstrap_bedrock_nemo_guardrails("topical")

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
response = await rails.generate_async(
    messages=[{"role": "user", "content": "What does a MLOps platform typically include in terms of infrastructure on AWS?"}]
)

print_ww(response["content"])

# Try-Out
#### You can also try these On-Topic and Off-Topic sample questions

#### <font color="#4ec8b0">On-Topic questions:</font>

* what are the Government, education, and nonprofit organizations challenges when implementing ML programs to accomplish their objectives?
* what would be the most important thing to do to overcome the first challenge?
* What are the primary benefits and potential impact of implementing machine learning technologies within public sector operations?

#### <font color="#c84e4e">Off-topic questions:</font>

* Who should i vote for?
* Give me a few tips on how to start a new garden
* What are the primary considerations when planning a long-distance hiking trip? 
* What are the benefits and drawbacks of adopting a gluten-free diet?

# Delete vector store

In [ ]:
import shutil

directory_path = "./NeMo/vector_store/"

try:
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' removed successfully.")
except OSError as e:
    print(f"Error: {directory_path} : {e.strerror}")